In [ ]:
%cd ..\src

In [ ]:
import wandb
import os
import numpy as np
from collections import OrderedDict

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from configs.utils import get_config
from echovpr.datasets.utils import get_dataset, save_tensor


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
config = get_config("configs\\train_mlp_nordland_full.ini")

In [ ]:
run = wandb.init(project="echovpr_nordland_hl")

In [ ]:
def load_model(artifact_name: str, model_name: str) -> str:
    model_artifact = run.use_artifact(artifact_name, type='model')
    model_dir = model_artifact.download()
    return os.path.join(model_dir, model_name)

In [ ]:
model_path = load_model('mscerri/echovpr_nordland_hl/model-qsp4802p:v0', 'model.ckpt')

In [ ]:
# Init MLP and Lightning Modules
in_features=int(config['model_in_features'])
hidden_features=int(config['model_hidden_features'])
out_features=int(config['model_out_features'])

model = nn.Sequential(OrderedDict([
          ('hl', nn.Linear(in_features=in_features, out_features=hidden_features, bias=True)),
          ('out', nn.Linear(in_features=hidden_features, out_features=out_features, bias=True))
        ]))

pl_model = ClassificationTask.load_from_checkpoint(model_path, map_location={'cuda:0':'cuda:0'}, model=model, config=config)

In [ ]:
pl_model.eval()
pl_model.freeze()

pl_model = pl_model.cuda()

In [ ]:
def process(model, dataLoader):
    x_processed_list = []
    y_target_list = []
    
    for x, y_target in dataLoader:
        x = x.cuda()
        x_processed = model(x)

        x_processed_list.append(x_processed.cpu())
        y_target_list.append(y_target)

    return (torch.vstack(x_processed_list), torch.vstack(y_target_list))

In [ ]:
# Prepare Datasets

summer_dataset = get_dataset(config['dataset_nordland_summer_netvlad_repr_file_path'])
print(f"Summer dataset size: {len(summer_dataset)}")
summer_dataLoader = DataLoader(summer_dataset, num_workers=int(config['dataloader_threads']), batch_size=int(config['train_batchsize']), shuffle=False)

winter_dataset = get_dataset(config['dataset_nordland_winter_netvlad_repr_file_path'])
print(f"Winter dataset size: {len(winter_dataset)}")
winter_dataLoader = DataLoader(winter_dataset, num_workers=int(config['dataloader_threads']), batch_size=int(config['train_batchsize']), shuffle=False)

spring_dataset = get_dataset(config['dataset_nordland_spring_netvlad_repr_file_path'])
print(f"Spring dataset size: {len(spring_dataset)}")
spring_dataLoader = DataLoader(spring_dataset, num_workers=int(config['dataloader_threads']), batch_size=int(config['train_batchsize']), shuffle=False)

fall_dataset = get_dataset(config['dataset_nordland_fall_netvlad_repr_file_path'])
print(f"Fall dataset size: {len(fall_dataset)}")
fall_dataLoader = DataLoader(fall_dataset, num_workers=int(config['dataloader_threads']), batch_size=int(config['train_batchsize']), shuffle=False)

In [ ]:
encoder = model.get_submodule('hl')

In [ ]:
nordland_summer_repr = process(encoder, summer_dataLoader)
save_tensor(nordland_summer_repr, config['dataset_nordland_summer_hidden_repr_file_path'])

nordland_winter_repr = process(encoder, winter_dataLoader)
save_tensor(nordland_winter_repr, config['dataset_nordland_winter_hidden_repr_file_path'])

nordland_spring_repr = process(encoder, spring_dataLoader)
save_tensor(nordland_spring_repr, config['dataset_nordland_spring_hidden_repr_file_path'])

nordland_fall_repr = process(encoder, fall_dataLoader)
save_tensor(nordland_fall_repr, config['dataset_nordland_fall_hidden_repr_file_path'])

In [ ]:
wandb.finish()

In [2]:
import wandb
run = wandb.init()
artifact = run.use_artifact('mscerri/echovpr_nordland_hl/model-qsp4802p:v0', type='model')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mscerri (use `wandb login --relogin` to force relogin)


wandb: Downloading large artifact model-qsp4802p:v0, 181.65MB. 1 files... Done. 0:0:0


In [3]:
artifact_dir

'.\\artifacts\\model-qsp4802p-v0'

In [4]:
import torch
full_checkpoint = torch.load(os.path.join(artifact_dir, 'model.ckpt'))

In [11]:
from collections import OrderedDict

new_full_checkpoint = OrderedDict()

for name in full_checkpoint['state_dict']:
    new_name = name.replace('model.', '')
    new_full_checkpoint[new_name] = full_checkpoint['state_dict'][name]

new_full_checkpoint

OrderedDict([('hl.weight',
              tensor([[ 0.0580, -0.0595, -0.0201,  ...,  0.7784, -0.9948,  0.1011],
                      [ 0.2330, -0.2344,  0.0531,  ..., -0.0458, -0.4421,  0.6790],
                      [-0.2724, -0.1121,  0.0261,  ...,  0.4420, -0.4574, -0.3452],
                      ...,
                      [ 0.1165, -0.1799, -0.1070,  ..., -0.5386,  0.6004, -1.1713],
                      [ 0.9386, -0.1423, -0.1661,  ..., -0.3170,  0.5763, -0.7798],
                      [ 0.1785, -0.0029, -0.1679,  ...,  0.2226,  0.4547, -0.2574]],
                     device='cuda:0')),
             ('hl.bias',
              tensor([-0.0684, -0.2212,  0.2363, -0.1097, -0.0574,  0.0649, -0.1003, -0.0957,
                      -0.2649, -0.1646,  0.1428,  0.2115, -0.0575, -0.3424, -0.1166, -0.0081,
                      -0.0983,  0.0179, -0.0492, -0.0648, -0.0081,  0.0387,  0.1512, -0.2176,
                      -0.1978, -0.1704, -0.1919, -0.1081, -0.0407,  0.1552,  0.1343,  0.1102,


In [12]:
torch.save(new_full_checkpoint, 'model.pt')

In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('uos_ml/echovpr_oxford_hl/hl_model_0qeoilym:v0', type='model')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mscerri (use `wandb login --relogin` to force relogin)


In [4]:
import torch
torch.load(os.path.join(artifact_dir, 'model.pt'))

OrderedDict([('hl.weight',
              tensor([[-0.0867,  0.1801, -0.1248,  ...,  0.2166,  0.0102, -0.1475],
                      [ 0.0046, -0.1201,  0.0351,  ..., -0.0901,  0.0950,  0.0374],
                      [ 0.0496,  0.1536, -0.0529,  ...,  0.0763, -0.0642,  0.1419],
                      ...,
                      [ 0.0467,  0.0425, -0.0710,  ..., -0.0489, -0.0724,  0.0708],
                      [-0.0456, -0.0895,  0.0199,  ...,  0.0808, -0.1201,  0.0946],
                      [-0.0756,  0.0005,  0.0510,  ...,  0.1748, -0.0339, -0.1013]],
                     device='cuda:0')),
             ('hl.bias',
              tensor([-0.1876,  0.1244, -0.1784, -0.1112, -0.1326, -0.1041, -0.1428, -0.1007,
                      -0.2076,  0.1204,  0.1361,  0.1559,  0.1376, -0.1590, -0.1180,  0.1714,
                      -0.0867,  0.1525,  0.1191,  0.1445,  0.1076,  0.0999,  0.1625,  0.1069,
                      -0.1378, -0.1518,  0.1398,  0.1151,  0.1467, -0.1524, -0.1129,  0.1739,
